In [12]:
#importing requests, BeautifulSoup, pandas, csv, wordcloud, matplotlib, stopwords
import nltk
import bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import DataFrame
import csv
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as mplot
from nltk.corpus import stopwords
stop = stopwords.words('english')
from textblob import Word
from textblob import TextBlob
import numpy as np
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\deepa\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [13]:
#reading the csv file

train = pd.read_csv('trustpilot.csv', index_col = 0)

In [14]:
pd.set_option('expand_frame_repr', False)
print(train)

                                                Title                                             Review
0                Business Owners AVOID at all costs..  American Express Open... openly takes full con...
1   Look elsewhere...Visa, MasterCard or Discover ...  So I requested a credit line increase because ...
2    AMERICAN EXPRESS - THE COMPANY YOU CAN'T RELY ON  Booked a holiday & paid by American Express fo...
3                              Never get an amex card  flights cancelled with airline.Been trying for...
4                     The Platinum Perennial Torture!  The Platinum Perennial Torture!Despite being A...
5                               Good customer support                  Great company. Resolved my issues
6                       Wonderful Customer Service!!!  I've never had customer service answer me so q...
7               I had a dispute opened with TravelUp.  I had a dispute opened with TravelUp.Whereas m...
8                                          The worst!  

# basic data features to know the details of the reviews

In [15]:
#Count of words
train['word_count'] = train['Review'].apply(lambda x: len(str(x).split(" ")))
#Count of characters
train['char_count'] = train['Review'].str.len() ## this also includes spaces
#Count of stopwords
train['stopwords'] = train['Review'].apply(lambda x: len([x for x in x.split() if x in stop]))
#count of numbers
train['numerics'] = train['Review'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

train[['Review','word_count','char_count','stopwords','numerics']].head()

,Review,word_count,char_count,stopwords,numerics
0,American Express Open... openly takes full con...,228,1291,98,1
1,So I requested a credit line increase because ...,111,580,45,0
2,Booked a holiday & paid by American Express fo...,58,339,19,1
3,flights cancelled with airline.Been trying for...,33,188,13,0
4,The Platinum Perennial Torture!Despite being A...,42,264,8,2


# pre-processing of the data

In [16]:
#converting to lowercase
train['Review'] = train['Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#removing punctuations
train['Review'] = train['Review'].str.replace('[^\w\s]','')
#removing stopwords
train['Review'] = train['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#lemmatization-it reduces the word to its root form
train['Review'] = train['Review'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
#Spelling correction
train['Review']=train['Review'].apply(lambda x: str(TextBlob(x).correct()))

train['Review'].head()

0    american express open openly take full control...
1    requested credit line increase american expres...
2    booked holiday paid american express nearly 52...
3    flight canceled airlinebeen trying week get re...
4    platino perennial torturedespite apex big plat...
Name: Review, dtype: object

In [17]:
#Term Frequency
tf1 = (train['Review']).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1

,words,tf
0,business,6.0
1,express,16.0
2,american,16.0
3,card,23.0
4,make,6.0
...,...,...
457,grace,1.0
458,find,1.0
459,speak,1.0
460,morning,1.0


In [9]:
#Inverse Document Frequency
for i,word in enumerate(tf1['words']):
  tf1.loc[i, 'idf'] = np.log(train.shape[0]/(len(train[train['reviews'].str.contains(word)])))
tf1

,words,tf,idf
0,point,203.0,0.000000
1,reward,130.0,0.154151
2,membership,79.0,0.154151
3,card,235.0,0.000000
4,11,13.0,1.252763
...,...,...,...
1198,bargain,1.0,2.639057
1199,correctly,1.0,2.639057
1200,012p,1.0,2.639057
1201,thatairlineguybusiness,1.0,2.639057


In [10]:
#tf-idf for each term
tf1['tfidf'] = tf1['tf'] * tf1['idf']
tf1

,words,tf,idf,tfidf
0,point,203.0,0.000000,0.000000
1,reward,130.0,0.154151,20.039588
2,membership,79.0,0.154151,12.177904
3,card,235.0,0.000000,0.000000
4,11,13.0,1.252763,16.285919
...,...,...,...,...
1198,bargain,1.0,2.639057,2.639057
1199,correctly,1.0,2.639057,2.639057
1200,012p,1.0,2.639057,2.639057
1201,thatairlineguybusiness,1.0,2.639057,2.639057


In [11]:
tf1.to_csv(r'C:\Users\cshashwa\Desktop\python\tfidf_uk_gold_expert.csv')